# Background

For the background assume I have a client who is looking to open a restaurant in San Francisco and needs to find the best possible places for it. The purpase of this project is to find the best area with the potential to open a tea garden mixed with a mederanian restaurant (this idea was conceived before the corona virus outbreak so please ignore the current market conditions for such establisments.) After acquirig the data for neigbourhood and clusturing them by their venues, I will try to find a cluster where the restaurants and food courts are concentrated but does not have many mediterenian restaurants and tea gardens. If such a cluster does not exist, I hope to find a cluster where the parks are concentrated.

# Data Acquisition and How It Will Be Used

The data will be acquired in three steps. These are:
1- Get the neigbourhoods in San Franscisco from Wikipedia, read and parse the data using beatifulsoup library and turn it into a dataframe.
2- Get the latitude and longititude information for these neighbourhoods using the geopy library
3- Get the top 10 venues for the neigbourhood using the Foursquare API


The above raw data will be turned into valuable information in two steps. These are:
1- After the data is acquired and cleaned use the scikit-learn library to cluster the data.
2- Use the folium library to create a visualization of the San Francisco and its neigboorhood clusters.

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Francisco").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    neighborhoodList.append(row.text)
cl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cl_df=cl_df.drop([0])
cl_df=cl_df.reset_index(drop=True)
#cl_df=cl_df.drop([95])

In [3]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Francisco, California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [4]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 8.8MB/s ta 0:00:011


In [5]:
coords = [ get_latlng(neighborhood) for neighborhood in cl_df["Neighborhood"].tolist() ]


In [6]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
cl_df['Latitude'] = df_coords['Latitude']
cl_df['Longitude'] = df_coords['Longitude']

In [7]:
cl_df.tail()

,Neighborhood,Latitude,Longitude
91,"West Portal, San Francisco",37.73990,-122.46749
92,"Western Addition, San Francisco",37.78095,-122.43222
93,"Westwood Highlands, San Francisco",37.73318,-122.45450
94,"Westwood Park, San Francisco",37.72813,-122.45816
95,Yerba Buena Island,37.81032,-122.36689


In [8]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of SF, CA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of SF, CA 37.7790262, -122.4199061.


In [9]:
CLIENT_ID = '2PHFHOLO0ZBGKXVRLDMIJBNSX4IFANVHCDJWX4RXYR2SHK4M' # your Foursquare ID
CLIENT_SECRET = 'N4CDMFJIVFMNSVVN2QAMTZJCCFT4PWKNQGDGA54YBUVZOMSH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2PHFHOLO0ZBGKXVRLDMIJBNSX4IFANVHCDJWX4RXYR2SHK4M
CLIENT_SECRET:N4CDMFJIVFMNSVVN2QAMTZJCCFT4PWKNQGDGA54YBUVZOMSH


In [10]:
LIMIT = 150 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2PHFHOLO0ZBGKXVRLDMIJBNSX4IFANVHCDJWX4RXYR2SHK4M&client_secret=N4CDMFJIVFMNSVVN2QAMTZJCCFT4PWKNQGDGA54YBUVZOMSH&v=20180605&ll=37.7790262,-122.4199061&radius=500&limit=150'

In [11]:
results = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(93, 4)

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        response= requests.get(url).json()["response"]
        if "groups" not in response:
            print(url)
            print(response)
            continue
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
nearby_venues.shape

(93, 4)

In [16]:
sf_venues = getNearbyVenues(names=cl_df['Neighborhood'],
                                   latitudes=cl_df['Latitude'],
                                   longitudes=cl_df['Longitude']
                                  )

Alamo Square, San Francisco
Alta Plaza
Anza Vista, San Francisco
Balboa Park, San Francisco
Balboa Terrace, San Francisco
Barbary Coast, San Francisco
Bayview–Hunters Point, San Francisco
Bayview, San Francisco, California
Bernal Heights, San Francisco
Carville, San Francisco
Castro District, San Francisco
Cathedral Hill, San Francisco
Central Embarcadero Piers Historic District
Central Sunset
Chinatown, San Francisco
Civic Center, San Francisco
Cole Valley, San Francisco
Compton's Transgender Cultural District
Corona Heights, San Francisco
Crocker-Amazon, San Francisco
Diamond Heights, San Francisco
Dogpatch, San Francisco
Duboce Park Landmark District
Duboce Triangle, San Francisco
Dumpville
Eureka Valley, San Francisco
Excelsior District, San Francisco
Fillmore District, San Francisco
Financial District, San Francisco
Fisherman's Wharf, San Francisco
Forest Hill, San Francisco
Forest Knolls, San Francisco
Glen Park, San Francisco
Haight-Ashbury
Hayes Valley, San Francisco
Hunters Po

In [17]:
print(sf_venues.shape)
sf_venues.head()

(4810, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Alamo Square, San Francisco",37.77722,-122.43146,Painted Ladies,37.776120,-122.433389,Historic Site
1,"Alamo Square, San Francisco",37.77722,-122.43146,Alamo Square,37.776045,-122.434363,Park
2,"Alamo Square, San Francisco",37.77722,-122.43146,Church of 8 Wheels,37.774733,-122.430862,Roller Rink
3,"Alamo Square, San Francisco",37.77722,-122.43146,The Center SF,37.774545,-122.430730,Spiritual Center
4,"Alamo Square, San Francisco",37.77722,-122.43146,Originals Vinyl,37.775835,-122.431227,Record Shop


In [18]:
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alamo Square, San Francisco",23,23,23,23,23,23
Alta Plaza,54,54,54,54,54,54
"Anza Vista, San Francisco",16,16,16,16,16,16
"Balboa Park, San Francisco",8,8,8,8,8,8
"Balboa Terrace, San Francisco",10,10,10,10,10,10
"Barbary Coast, San Francisco",70,70,70,70,70,70
"Bayview, San Francisco, California",37,37,37,37,37,37
"Bayview–Hunters Point, San Francisco",5,5,5,5,5,5
"Bernal Heights, San Francisco",35,35,35,35,35,35


In [19]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 348 uniques categories.


In [20]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Yoga Studio,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Gym,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,

In [21]:
sf_onehot.shape

(4810, 348)

In [22]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped

,Neighborhood,Yoga Studio,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Gym,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,

In [23]:
sf_grouped.shape

(96, 348)

In [24]:
num_top_venues = 5

for hood in sf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alamo Square, San Francisco----
               venue  freq
0        Roller Rink  0.04
1     Ice Cream Shop  0.04
2  German Restaurant  0.04
3   Spiritual Center  0.04
4   Sushi Restaurant  0.04


----Alta Plaza----
                venue  freq
0  Chinese Restaurant  0.06
1                Park  0.06
2      Cosmetics Shop  0.06
3       Grocery Store  0.04
4                 Spa  0.04


----Anza Vista, San Francisco----
                     venue  freq
0                     Café  0.19
1              Coffee Shop  0.06
2  Health & Beauty Service  0.06
3      Arts & Crafts Store  0.06
4       Mexican Restaurant  0.06


----Balboa Park, San Francisco----
                    venue  freq
0            Tennis Court  0.12
1           Metro Station  0.12
2  Furniture / Home Store  0.12
3            Dessert Shop  0.12
4          Sandwich Place  0.12


----Balboa Terrace, San Francisco----
                venue  freq
0  Light Rail Station   0.2
1            Pharmacy   0.1
2        Intersection   0.

                 venue  freq
0  Japanese Restaurant  0.04
1           Food Truck  0.04
2          Men's Store  0.04
3          Coffee Shop  0.03
4                 Café  0.03


----Japantown, San Francisco----
           venue  freq
0       Tea Room  0.05
1         Bakery  0.05
2      Gift Shop  0.05
3  Grocery Store  0.04
4  Shopping Mall  0.04


----Laurel Heights, San Francisco----
                     venue  freq
0   Furniture / Home Store  0.06
1           Cosmetics Shop  0.06
2  Health & Beauty Service  0.04
3              Coffee Shop  0.04
4                     Café  0.04


----Lincoln Manor----
            venue  freq
0  Scenic Lookout  0.14
1    Fishing Spot  0.07
2  Baseball Field  0.07
3           Trail  0.07
4           Beach  0.07


----Little Hollywood, San Francisco----
                 venue  freq
0        Train Station  0.14
1  American Restaurant  0.14
2          Coffee Shop  0.14
3       Sandwich Place  0.14
4   Light Rail Station  0.14


----Lone Mountain (California

          venue  freq
0         Hotel  0.07
1      Boutique  0.05
2          Café  0.04
3  Cocktail Bar  0.04
4     Speakeasy  0.03


----Treasure Island, San Francisco----
                venue  freq
0          Food Truck   0.3
1      Breakfast Spot   0.1
2                 Gym   0.1
3    Video Game Store   0.1
4  Athletics & Sports   0.1


----Twin Peaks (San Francisco)----
            venue  freq
0  Scenic Lookout  0.18
1           Trail  0.18
2     Yoga Studio  0.09
3            Park  0.09
4       Speakeasy  0.09


----Union Square, San Francisco----
                  venue  freq
0                 Hotel  0.11
1              Boutique  0.05
2  Gym / Fitness Center  0.04
3        Clothing Store  0.04
4                  Café  0.03


----Upper Fillmore----
           venue  freq
0      Gift Shop  0.05
1       Tea Room  0.05
2         Bakery  0.05
3  Grocery Store  0.04
4  Shopping Mall  0.04


----Visitacion Valley, San Francisco----
                   venue  freq
0           Home Servic

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alamo Square, San Francisco",Park,Dog Run,Spa,Museum,Food Truck,French Restaurant,Bus Line,Sushi Restaurant,Café,German Restaurant
1,Alta Plaza,Cosmetics Shop,Park,Chinese Restaurant,Salon / Barbershop,Grocery Store,Pizza Place,Furniture / Home Store,Spa,Sandwich Place,Bakery
2,"Anza Vista, San Francisco",Café,Sandwich Place,Big Box Store,Donut Shop,Coffee Shop,Grocery Store,Tunnel,Arts & Crafts Store,Cosmetics Shop,Bus Line
3,"Balboa Park, San Francisco",Furniture / Home Store,Pool,Metro Station,Tennis Court,Coffee Shop,Sandwich Place,Park,Dessert Shop,Food Stand,Falafel Restaurant
4,"Balboa Terrace, San Francisco",Light Rail Station,Vietnamese Restaurant,Playground,Fountain,Intersection,Park,Pharmacy,Gym,Comic Shop,Exhibit


In [27]:
# set number of clusters
kclusters = 10

sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([3, 5, 0, 5, 5], dtype=int32)

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

sf_merged = cl_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#sf_merged['Cluster Labels']=sf_merged['Cluster Labels'].astype(str)
sf_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alamo Square, San Francisco",37.77722,-122.43146,3,Park,Dog Run,Spa,Museum,Food Truck,French Restaurant,Bus Line,Sushi Restaurant,Café,German Restaurant
1,Alta Plaza,37.79101,-122.43930,5,Cosmetics Shop,Park,Chinese Restaurant,Salon / Barbershop,Grocery Store,Pizza Place,Furniture / Home Store,Spa,Sandwich Place,Bakery
2,"Anza Vista, San Francisco",37.78048,-122.44358,0,Café,Sandwich Place,Big Box Store,Donut Shop,Coffee Shop,Grocery Store,Tunnel,Arts & Crafts Store,Cosmetics Shop,Bus Line
3,"Balboa Park, San Francisco",37.72493,-122.44314,5,Furniture / Home Store,Pool,Metro Station,Tennis Court,Coffee Shop,Sandwich Place,Park,Dessert Shop,Food Stand,Falafel Restaurant
4,"Balboa Terrace, San Francisco",37.73180,-122.46740,5,Light Rail Station,Vietnamese Restaurant,Playground,Fountain,Intersection,Park,Pharmacy,Gym,Comic Shop,Exhibit


In [57]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Anza Vista, San Francisco",Café,Sandwich Place,Big Box Store,Donut Shop,Coffee Shop,Grocery Store,Tunnel,Arts & Crafts Store,Cosmetics Shop,Bus Line
12,Central Embarcadero Piers Historic District,Gym,Sandwich Place,Scenic Lookout,American Restaurant,Mexican Restaurant,Burger Joint,Café,Lounge,Dog Run,Park
27,"Fillmore District, San Francisco",Baseball Stadium,Coffee Shop,Café,New American Restaurant,Mexican Restaurant,Gym,Sandwich Place,Park,Brewery,Bar
46,"Little Hollywood, San Francisco",Train Station,Café,American Restaurant,Mountain,Coffee Shop,Light Rail Station,Sandwich Place,Flower Shop,Falafel Restaurant,Farmers Market
52,"Merced Manor, San Francisco",Food Truck,Gym,Playground,Sandwich Place,Music Venue,Food Stand,Weight Loss Center,High School,Farmers Market,Tennis Court
53,"Mid-Market, San Francisco",Coffee Shop,Food Truck,Gym / Fitness Center,Seafood Restaurant,Gym,Mediterranean Restaurant,Hotel,Burger Joint,Cocktail Bar,Café
54,"Mission Bay, San Francisco",Food Truck,Coffee Shop,Park,Café,Sandwich Place,Pharmacy,Street Food Gathering,Bar,New American Restaurant,Pool
66,"Parkmerced, San Francisco",Park,Gym,Dog Run,Candy Store,Church,Cocktail Bar,Art Gallery,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
73,"Rincon Hill, San Francisco",Café,Gym,Coffee Shop,Spa,American Restaurant,Mediterranean Restaurant,Yoga Studio,Pet Store,Brewery,Burger Joint
75,San Francisco Residence Parks,Hotel,Gym / Fitness Center,Café,Bridge,Hotel Bar,State / Provincial Park,Food Truck,American Restaurant,Gym,Coffee Shop


In [46]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,"Forest Knolls, San Francisco",Trail,Playground,Park,Garden,Tennis Court,Flea Market,Exhibit,Fried Chicken Joint,Falafel Restaurant,Farmers Market


In [47]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,"Visitacion Valley, San Francisco",Park,Home Service,Coffee Shop,Grocery Store,Vietnamese Restaurant,Women's Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market


In [48]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alamo Square, San Francisco",Park,Dog Run,Spa,Museum,Food Truck,French Restaurant,Bus Line,Sushi Restaurant,Café,German Restaurant
5,"Barbary Coast, San Francisco",Coffee Shop,Hotel,Vietnamese Restaurant,Bakery,Marijuana Dispensary,Gym / Fitness Center,Music Venue,Burger Joint,Cocktail Bar,Sandwich Place
8,"Bernal Heights, San Francisco",Coffee Shop,Italian Restaurant,Gourmet Shop,Bakery,Playground,Park,Trail,Sandwich Place,Indian Restaurant,Bus Stop
9,"Carville, San Francisco",Coffee Shop,Wine Bar,Dessert Shop,Theater,Café,Clothing Store,Cocktail Bar,Marijuana Dispensary,New American Restaurant,Concert Hall
11,"Cathedral Hill, San Francisco",Hotel,Italian Restaurant,Café,Bar,Spa,Breakfast Spot,Cocktail Bar,Beer Bar,Sushi Restaurant,American Restaurant
14,"Chinatown, San Francisco",Coffee Shop,Chinese Restaurant,Italian Restaurant,Cocktail Bar,Men's Store,Bakery,Hotel,Café,Deli / Bodega,Wine Bar
15,"Civic Center, San Francisco",Coffee Shop,Theater,Beer Bar,Marijuana Dispensary,Dance Studio,American Restaurant,Concert Hall,Cocktail Bar,Hotel Bar,Café
16,"Cole Valley, San Francisco",Wine Bar,Breakfast Spot,Bus Station,Yoga Studio,Coffee Shop,Tapas Restaurant,Taco Place,Board Shop,Bookstore,Sports Club
17,Compton's Transgender Cultural District,Coffee Shop,Wine Bar,Dessert Shop,Theater,Café,Clothing Store,Cocktail Bar,Marijuana Dispensary,New American Restaurant,Concert Hall
21,"Dogpatch, San Francisco",Gym / Fitness Center,Café,Wine Bar,Food Truck,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Gift Shop,Cocktail Bar


In [49]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Castro District, San Francisco",Gay Bar,Thai Restaurant,Coffee Shop,Scenic Lookout,Yoga Studio,Arts & Crafts Store,Playground,Deli / Bodega,Convenience Store,Indian Restaurant
18,"Corona Heights, San Francisco",Gay Bar,Park,Scenic Lookout,Monument / Landmark,Yoga Studio,Pizza Place,Supermarket,Shoe Store,Coffee Shop,Salon / Barbershop
19,"Crocker-Amazon, San Francisco",Scenic Lookout,Gastropub,Liquor Store,Dog Run,Playground,Women's Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
20,"Diamond Heights, San Francisco",Bus Station,Trail,Grocery Store,Bakery,Dog Run,Athletics & Sports,Scenic Lookout,Coffee Shop,Pharmacy,Salon / Barbershop
25,"Eureka Valley, San Francisco",Gay Bar,Scenic Lookout,Thai Restaurant,Convenience Store,Coffee Shop,Yoga Studio,Wine Bar,Playground,Bus Station,Café
61,"Oceanview, San Francisco",Playground,Grocery Store,Thai Restaurant,Food,Liquor Store,Home Service,Coffee Shop,Light Rail Station,Fishing Spot,Falafel Restaurant
70,Presidio Terrace,Golf Course,American Restaurant,Yoga Studio,Coffee Shop,Burmese Restaurant,Other Nightlife,Sculpture Garden,Liquor Store,Scenic Lookout,Vegetarian / Vegan Restaurant
87,Twin Peaks (San Francisco),Trail,Scenic Lookout,Playground,Park,Hill,Café,Grocery Store,Yoga Studio,Speakeasy,Fondue Restaurant


I chose cluster 5 to be the best cluster to open a tea garden and a Mediterranean restaurant

In [50]:
sf_merged.loc[sf_merged['Cluster Labels'] == 5, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Alta Plaza,Cosmetics Shop,Park,Chinese Restaurant,Salon / Barbershop,Grocery Store,Pizza Place,Furniture / Home Store,Spa,Sandwich Place,Bakery
3,"Balboa Park, San Francisco",Furniture / Home Store,Pool,Metro Station,Tennis Court,Coffee Shop,Sandwich Place,Park,Dessert Shop,Food Stand,Falafel Restaurant
4,"Balboa Terrace, San Francisco",Light Rail Station,Vietnamese Restaurant,Playground,Fountain,Intersection,Park,Pharmacy,Gym,Comic Shop,Exhibit
7,"Bayview, San Francisco, California",Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Grocery Store,Dim Sum Restaurant,Bus Station,Bakery,Sandwich Place,Dessert Shop,Coffee Shop
13,Central Sunset,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Bar,Bank,Coffee Shop,Pharmacy,Sandwich Place,Donut Shop
32,"Glen Park, San Francisco",Trail,Dog Run,Sushi Restaurant,Bookstore,Gift Shop,Gym,Park,Cheese Shop,Chinese Restaurant,Bakery
37,"Ingleside Terraces, San Francisco",Playground,Chinese Restaurant,Yoga Studio,Café,Gym / Fitness Center,Noodle House,Park,Pool Hall,Convenience Store,Construction & Landscaping
67,"Parkside, San Francisco",Chinese Restaurant,Dumpling Restaurant,Sandwich Place,Park,Sushi Restaurant,Café,Dog Run,Music Store,Burrito Place,Pizza Place
68,Potrero Hill,Park,Grocery Store,Convenience Store,Hill,Café,Liquor Store,Bus Stop,Plaza,Japanese Restaurant,Playground
71,Rancho Las Camaritas,Fast Food Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Shipping Store,Grocery Store,Mexican Restaurant,Health & Beauty Service,Campground,Café


In [51]:
sf_merged.loc[sf_merged['Cluster Labels'] == 6, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Bayview–Hunters Point, San Francisco",Park,Brewery,Waterfront,Coffee Shop,Women's Store,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
35,"Hunters Point, San Francisco, California",Park,Brewery,Waterfront,Coffee Shop,Women's Store,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [52]:
sf_merged.loc[sf_merged['Cluster Labels'] == 7, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,"Forest Hill, San Francisco",Park,Japanese Restaurant,Tennis Court,Playground,Hotpot Restaurant,French Restaurant,Food Stand,Falafel Restaurant,Fried Chicken Joint,Farmers Market
57,Mount Davidson (California),Park,Monument / Landmark,Bus Line,Playground,Women's Store,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot


In [53]:
sf_merged.loc[sf_merged['Cluster Labels'] == 8, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Lincoln Manor,Scenic Lookout,Baseball Field,Beach,Trail,Fishing Spot,Tourist Information Center,Historic Site,Sculpture Garden,Toll Plaza,Science Museum
95,Yerba Buena Island,Beach,Scenic Lookout,Park,Island,Tunnel,Diner,Bike Trail,History Museum,Women's Store,Farmers Market


In [54]:
sf_merged.loc[sf_merged['Cluster Labels'] == 9, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,"Treasure Island, San Francisco",Food Truck,Breakfast Spot,American Restaurant,Athletics & Sports,Gym,Video Game Store,Baseball Field,Island,Fried Chicken Joint,Fondue Restaurant


In [55]:
sf_merged.loc[sf_merged['Cluster Labels'] == 10, sf_merged.columns[[0] + list(range(4, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


I use 10 clusters because less than this number does not let me differentiate within the urban areas. Clustering has been very succesful in grouping sub-urban and urban areas and very succesful in grouping various different types of sub urban areas such as ones that are mainly used for outings with places like parks, or areas for socializing such as places with more restaurant and cafes. With 10 clusters the urban areas have also been grouped further similar to